<a href="https://colab.research.google.com/github/TienLungSun/LLM/blob/main/Video%20process%20examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: python code example at https://huggingface.co/allenai/Molmo-7B-D-0924

In [1]:
from PIL import Image

In [2]:
def form_large_img(frameLst):
  # 載入圖像
  images = [Image.open(img) for img in frameLst]

  # 假設所有圖片大小相同，取得圖片的寬高
  width, height = images[0].size

  # 設定拼接後的圖片大小 (這裡以3x3排列為例)
  new_width = width * 3
  new_height = height * 3

  # 建立一個空的畫布
  new_image = Image.new('RGB', (new_width, new_height))

  # 將每張圖片貼到大圖上
  for i, image in enumerate(images):
      # 計算要貼上的位置 (行列排列)
      x = (i % 3) * width
      y = (i // 3) * height
      new_image.paste(image, (x, y))

  # 儲存拼接後的圖像
  return new_image

In [5]:
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
import time

# 設定要讀取的視頻文件路徑
video_path = "entrance_mo.mp4"

# 使用 OpenCV 讀取視頻
cap = cv2.VideoCapture(video_path)

# 檢查是否成功打開視頻文件
if not cap.isOpened():
    print("無法開啟視頻文件")
else:
    frameNo =1
    frameLst = []
    while cap.isOpened():
        # 逐幀讀取視頻
        ret, frame = cap.read()
        frameNo +=1
        # 如果成功讀取到幀
        if ret:
            # 清除先前顯示的幀
            clear_output(wait=True)

            # 縮小顯示大小，比如設置為原來的 25%
            resized_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

            if(len(frameLst)<9):
              frameLst.append(resized_frame)
            else: #we already have 9 frmaes
              frameLst.pop(0)
              frameLst.append(resized_frame)
              new_img = form_large_img(frameLst)

              # 顯示當前幀
              resized_new_img = cv2.resize(new_img, (0, 0), fx=0.25, fy=0.25)
              cv2_imshow(resized_new_img)
              print("frame No:", frameNo)

              # 控制播放速度 (例如每秒25幀)
              time.sleep(0.1)  # 0.04 秒大約等於 25 fps
        else:
            # 如果無法讀取到幀，則結束循環
            break

# 釋放視頻對象
cap.release()

# 關閉所有窗口
cv2.destroyAllWindows()


無法開啟視頻文件
